# Imports

In [1]:
import config

In [2]:
import riotwatcher
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
def scrape_player_data():
    """
    By taking in one summoner and using their past tft games to spider into other user IDs,
    using their match histories to find more users and more matches.
    
    [TO-DO] 
    ENSURE UNIQUE MATCH IDs (No Repeats)
    ENSURE UNIQUE SUMMONER IDs (No Traceback)
    
    LIMIT = 100 REQUESTS/2MIN
            20 REQUESTS/1SEC
    
    INIT CENTER = 1 REQ
    
    1 ROTATION = 3 REQS
    MATCH LIST = 1 REQ
    PARTICIPANTS = 1 REQ
    NEW CENTER = 1 REQ
    
    33 CYCLES EVERY 2 MIN
    
    SCRAPE PLAYERS->AND THEN GET MATCHES
    
    """
    
    #Initial Point
    

In [43]:
def check_rate(rate, limit):
    if rate > limit-10:
        time.sleep(120)

In [44]:
def scrape_player_data(size):
    """
    Scrapes from an intial summoner, gathering summoner puuIDs in a set (to avoid duplicates)
    until set is equal to size in size.
    
    LOGIC:
    GET INIT
    GET MATCH
    GET PARTICIPANTS
    ADD UNIQUE PARTICIPANTS TO UNIQUE_CHAIN
    ADD PARTICIPANTS TO SUMMONER_SET
    FROM UNIQUE PARTICIPANTS GET MATCH
    GET PARTICIPANTS
    """
    

    #Sets up API
    tft = riotwatcher.TftWatcher(config.api_key)
    #GET INIT
    initial_summoner = tft.summoner.by_name('na1', config.initial_summoner)
    
    #Return object
    summoner_list = [initial_summoner['puuid']]
    
    while(len(summoner_list) < size):
        
        #counts number of requests made to API
        #limit is 100/2min
        rate = 0
        
        #Current Summoner is puuID
        current_summoner = summoner_list[-1]
        current_match_id_list = tft.match.by_puuid('americas',current_summoner, count=2)
        
        rate +=1
        check_rate(rate, 200)
        
        for current_match_id in current_match_id_list:
            current_match = tft.match.by_id('americas',current_match_id)
            
            rate +=1
            check_rate(rate,200)
            
            for participant_puuid in current_match['metadata']['participants']:
                #New puuID
                if participant_puuid not in summoner_list:
                    pass
    
    return summoner_list

In [45]:
player_list = scrape_player_data(10_000) 

In [46]:
len(player_list) 

10002

In [48]:
summoner_puuids = pd.DataFrame(player_list, columns = ['summoner_puuid'])

In [49]:
summoner_puuids.to_csv('../datasets/summoner_puuids.csv',index=False)

In [10]:
tft = riotwatcher.TftWatcher(config.api_key)

In [11]:
initial_summoner = tft.summoner.by_name('na1', config.initial_summoner)

In [12]:
match_list = tft.match.by_puuid('americas',initial_summoner['puuid'])

In [13]:
sample_match = tft.match.by_id('americas',match_list[0])

In [14]:
sample_match['metadata']['participants']

['uYusCxO1ixEhomxcoiioMPfLebToRRcSKMt7xY39wE3V53KLbRGdhn_AfN9sYr3u8tukkFS0MDuF7w',
 'Kd4Xz5DXbrEphO15MDzByPFqYdWVMT_L27vH7Yq6DfDn66v3S9eEBV6Lb0hky-wrxLZHH99taPKkjg',
 'RSs0bQs81xWzeL81iClAevfOirVGPBE4Ctf0lymmscxjSQN4IuoThcaSDdg1NZOic4om3HlUztIvzg',
 'mBsMqJuSWqnxxLUGdPTiiHqDivZNrh64n3MND65hu--cV9iu-DWOZfpQ6fSQI78v95FG6GStveMfMg',
 'Mm4mqIDNu7szCdtub6foYrxxIWtvKLI_Skkhev40_nwN2_my33OgChdFzYr4pr0oknU_GUxEtAX9KQ',
 'mh5Nb6lS5oh7DMQx3GP2Qi75BDno9jxTTeNarIuXbZtDFrkVebcv0mHG8drTNqW9pkS3j3CjynaYuQ',
 'uck8AyTmr1dOrD-URr8DN67mu3uACQ9XyC1a7u7JOX7OtL1u2da1NyqvQv5Ans48OwxOPSh_qQljuA',
 'wXOtnYXjR1PAkmI9OHu4ry_ZX5UdRwkK0dGFZzs4EpHYCs02LKJhzlNUe9EQWkUnLEX7qW0VJNIqCg']

In [15]:
match_list

['NA1_4175441750',
 'NA1_4175344199',
 'NA1_4175192906',
 'NA1_4173918564',
 'NA1_4173627832',
 'NA1_4173393611',
 'NA1_4172894802',
 'NA1_4172372771',
 'NA1_4168292412',
 'NA1_4167869305',
 'NA1_4167767160',
 'NA1_4166927431',
 'NA1_4166765192',
 'NA1_4166731956',
 'NA1_4166717735',
 'NA1_4166675579',
 'NA1_4165890971',
 'NA1_4165614472',
 'NA1_4164205965',
 'NA1_4164149583']